In [1]:
import pandas as pd
import numpy as np

# 01 - Data Ingestion & Preprocessing

This notebook loads and combines raw Fannie Mae Credit Risk Transfer (CRT) data across multiple time periods. The data is pipe-delimited and requires header handling from a separate file.

## Objectives:
- Load CRT loan data across several months
- Apply the correct headers from the provided reference file
- Concatenate data across time into a master dataset
- Create the binary target variable `prepay`
- Convert string dates to datetime format for time-based grouping


In [ ]:
file_loc='/Users/Downloads/CAS-102013-082023'
df1=pd.read_csv(file_loc+'/CAS_2018-C01_G1_012018.csv', delimiter='|', header=None)

In [13]:
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,98,99,100,101,102,103,104,105,106,107
0,1501,90000001,112017,C,"Flagstar Bank, FSB","Flagstar Bank, FSB",FANNIE MAE,4.375,4.375,232000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1501,90000002,112017,R,Other,Matrix Financial Services Corporation,FANNIE MAE,4.000,4.000,236000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1501,90000003,112017,C,Other,Other,FANNIE MAE,4.500,4.500,95000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1501,90000004,112017,R,"Movement Mortgage, LLC",Matrix Financial Services Corporation,FANNIE MAE,4.625,4.625,388000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1501,90000005,112017,C,"Wells Fargo Bank, N.A.","Wells Fargo Bank, N.A.",FANNIE MAE,4.500,4.500,248000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
header=pd.read_csv(file_loc+'/CRT_Header_File.csv')

In [25]:
header

,Reference Pool ID,Loan Identifier,Monthly Reporting Period,Channel,Seller Name,Servicer Name,Master Servicer,Original Interest Rate,Current Interest Rate,Original UPB,...,Mortgage Margin,ARM Balloon Indicator,ARM Plan Number,Borrower Assistance Plan,High Loan to Value (HLTV) Refinance Option Indicator,Deal Name,Repurchase Make Whole Proceeds Flag,Alternative Delinquency Resolution,Alternative Delinquency Resolution Count,Total Deferral Amount


### ### Apply Headers to the Sample

Column names are applied from a separate header file provided by Fannie Mae.

In [15]:
df1.columns=header.columns

In [ ]:
df1.head()

,Reference Pool ID,Loan Identifier,Monthly Reporting Period,Channel,Seller Name,Servicer Name,Master Servicer,Original Interest Rate,Current Interest Rate,Original UPB,...,Mortgage Margin,ARM Balloon Indicator,ARM Plan Number,Borrower Assistance Plan,High Loan to Value (HLTV) Refinance Option Indicator,Deal Name,Repurchase Make Whole Proceeds Flag,Alternative Delinquency Resolution,Alternative Delinquency Resolution Count,Total Deferral Amount
0,1501,90000001,112017,C,"Flagstar Bank, FSB","Flagstar Bank, FSB",FANNIE MAE,4.375,4.375,232000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1501,90000002,112017,R,Other,Matrix Financial Services Corporation,FANNIE MAE,4.000,4.000,236000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1501,90000003,112017,C,Other,Other,FANNIE MAE,4.500,4.500,95000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1501,90000004,112017,R,"Movement Mortgage, LLC",Matrix Financial Services Corporation,FANNIE MAE,4.625,4.625,388000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1501,90000005,112017,C,"Wells Fargo Bank, N.A.","Wells Fargo Bank, N.A.",FANNIE MAE,4.500,4.500,248000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
header=pd.read_csv(file_loc+'/CRT_Header_File.csv')
for i in range(1,7):
    print(i)
    filename = '/CAS_2018-C01_G1_'+str(i).zfill(2)+'2018.csv'
    df1=pd.read_csv(file_loc+filename, delimiter='|', header=None)
    df1.columns=header.columns
    header = pd.concat([header,df1])

1
2
3


/var/folders/nb/cq9dyksn0z39zq6k8k66f0xc0000gn/T/ipykernel_29552/1736986010.py:5: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  df1=pd.read_csv(file_loc+filename, delimiter='|', header=None)


4


/var/folders/nb/cq9dyksn0z39zq6k8k66f0xc0000gn/T/ipykernel_29552/1736986010.py:5: DtypeWarning: Columns (42,101) have mixed types. Specify dtype option on import or set low_memory=False.
  df1=pd.read_csv(file_loc+filename, delimiter='|', header=None)


5


/var/folders/nb/cq9dyksn0z39zq6k8k66f0xc0000gn/T/ipykernel_29552/1736986010.py:5: DtypeWarning: Columns (42,101) have mixed types. Specify dtype option on import or set low_memory=False.
  df1=pd.read_csv(file_loc+filename, delimiter='|', header=None)


6


/var/folders/nb/cq9dyksn0z39zq6k8k66f0xc0000gn/T/ipykernel_29552/1736986010.py:5: DtypeWarning: Columns (42,101) have mixed types. Specify dtype option on import or set low_memory=False.
  df1=pd.read_csv(file_loc+filename, delimiter='|', header=None)


In [32]:
header.groupby('Monthly Reporting Period').size()

Monthly Reporting Period
12018     186525
22018     186525
32018     186525
42018     186525
112017    186525
122017    186525
dtype: int64

### Create Prepayment Flag

Defining a binary target `prepay` = 'Y' if Zero Balance Code equals 1 (prepaid), otherwise 'N'.

In [37]:
header['prepay']="N"
header.loc[header['Zero Balance Code'] == 1, 'prepay'] = "Y"
header['Zero Balance Code'].dtype

dtype('float64')

In [39]:
header.groupby(['Monthly Reporting Period','prepay']).size()

Monthly Reporting Period  prepay
12018                     N         184318
                          Y           2207
22018                     N         183441
                          Y           3084
32018                     N         182416
                          Y           4109
42018                     N         181498
                          Y           5027
112017                    N         186525
122017                    N         185392
                          Y           1133
dtype: int64

### Convert Monthly Reporting Period to DateTime

To enable time-based analysis, the `Monthly Reporting Period` is converted into datetime format.

In [40]:
header['reporting_period'] = pd.to_datetime(header['Monthly Reporting Period'].astype(str), format='%m%Y')

In [41]:
header['reporting_period'].head()

0   2017-11-01
1   2017-11-01
2   2017-11-01
3   2017-11-01
4   2017-11-01
Name: reporting_period, dtype: datetime64[ns]

In [42]:
header.groupby(['reporting_period','prepay']).size()

reporting_period  prepay
2017-11-01        N         186525
2017-12-01        N         185392
                  Y           1133
2018-01-01        N         184318
                  Y           2207
2018-02-01        N         183441
                  Y           3084
2018-03-01        N         182416
                  Y           4109
2018-04-01        N         181498
                  Y           5027
dtype: int64

# Todo 

1. get more data in
2. start to explore relationship between prepay and other predictor variables: correlation, bin plot Current Actual UPB: divide UPB by 0 to100K, 100k to 200k, 200k to 300k, 300k+, plot the binned UPB against prepay.
3. get interest rate data: get 10 year treasury rate from 2017 to 2023
4. produce prepay percentage in groupby statement